## Temporal landing

In [1]:
#!pip install datasets

In [2]:
# Importing useful dependencies
import io
import ast
import boto3
import requests
from datasets import load_dataset


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\SakuraSnow\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\SakuraSnow\AppData\Local\Programs\Python\Python311\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\SakuraSnow\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\

AttributeError: _ARRAY_API not found

In [3]:
# Setup S3 client for MinIO (MinIO implements Amazon S3 API)
s3 = boto3.client(
    "s3",
    endpoint_url="http://127.0.0.1:9000", # MinIO API endpoint
    aws_access_key_id="minioadmin", # User name
    aws_secret_access_key="minioadmin", # Password
)

In [4]:
# We are going to use two datasets, one from: https://huggingface.co/datasets/FronkonGames/steam-games-dataset (123 MB)
ds1_raw = load_dataset("FronkonGames/steam-games-dataset")
# The other one from: https://huggingface.co/datasets/atalaydenknalbant/rawg-games-dataset (998 MB)
ds2_raw = load_dataset("atalaydenknalbant/rawg-games-dataset")

# Print the number of rows of each dataset
print(f"The dataset 1 contains {ds1_raw['train'].num_rows} rows")
print(f"The dataset 2 contains {ds2_raw['train'].num_rows} rows")

README.md: 0.00B [00:00, ?B/s]

C:\Users\SakuraSnow\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SakuraSnow\.cache\huggingface\hub\datasets--FronkonGames--steam-games-dataset. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not

(…)-00000-of-00001-e2ed184370a06932.parquet:   0%|          | 0.00/123M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/83560 [00:00<?, ? examples/s]

README.md: 0.00B [00:00, ?B/s]

C:\Users\SakuraSnow\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SakuraSnow\.cache\huggingface\hub\datasets--atalaydenknalbant--rawg-games-dataset. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is

rawg-games-dataset.parquet:   0%|          | 0.00/995M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/889793 [00:00<?, ? examples/s]

The dataset 1 contains 83560 rows
The dataset 2 contains 889793 rows


In [5]:
# We are going to use the first 500 obs from each dataset for testing purposes
ds1 = ds1_raw['train'][0:500]
ds2 = ds2_raw['train'][0:500]

# Print the number of rows of each subdataset
print(f"The subdataset 1 contains {len(ds1['About the game'])} rows")
print(f"The subdataset 2 contains {len(ds2['description'])} rows")

The subdataset 1 contains 500 rows
The subdataset 2 contains 500 rows


In [6]:
# We are interested on Text, Image and Video data
# We can find each of these data in the following columns
# ds1: "About the game" (Text), "Header image" (Image), "Screenshots" (Image), "Movies" (Video)
# ds2: "description" (Text), "background_image" (Image), "background_image_additional" (Image), "short_screenshots" (Image)
# By combing both datasets, we assume there will be duplicates of games

**Uploading Texts**

In [7]:
# This function uploads each game description from strings to the given bucket_name,
# saving them as separate text files (text_1.txt, text_2.txt, …).
def upload_strings_separately(bucket_name, client, strings, path="temporal-landing/", prefix="text"):
    for i, s in enumerate(strings, start=1):
        if not s: # skip empty strings or None
            continue
        object_name = f"{path}{prefix}_{i}.txt" # temporal-landing/text_1.txt, temporal-landing/text_2.txt ...
        client.put_object(
            Bucket=bucket_name,
            Key=object_name,
            Body=io.BytesIO(s.encode("utf-8")),
            ContentType="text/plain"
        )

        print(f"Uploaded: {object_name}")

# Uploading text files (combining both datasets)
upload_strings_separately("landing-zone", s3, strings = 
                          ds1['About the game'] +
                          ds2['description'],
                          path = "temporal-landing/")

Uploaded: temporal-landing/text_1.txt
Uploaded: temporal-landing/text_2.txt
Uploaded: temporal-landing/text_3.txt
Uploaded: temporal-landing/text_4.txt
Uploaded: temporal-landing/text_5.txt
Uploaded: temporal-landing/text_6.txt
Uploaded: temporal-landing/text_7.txt
Uploaded: temporal-landing/text_8.txt
Uploaded: temporal-landing/text_9.txt
Uploaded: temporal-landing/text_10.txt
Uploaded: temporal-landing/text_11.txt
Uploaded: temporal-landing/text_12.txt
Uploaded: temporal-landing/text_13.txt
Uploaded: temporal-landing/text_14.txt
Uploaded: temporal-landing/text_15.txt
Uploaded: temporal-landing/text_16.txt
Uploaded: temporal-landing/text_17.txt
Uploaded: temporal-landing/text_18.txt
Uploaded: temporal-landing/text_19.txt
Uploaded: temporal-landing/text_20.txt
Uploaded: temporal-landing/text_21.txt
Uploaded: temporal-landing/text_22.txt
Uploaded: temporal-landing/text_23.txt
Uploaded: temporal-landing/text_24.txt
Uploaded: temporal-landing/text_25.txt
Uploaded: temporal-landing/text_26

**Uploading Images**

In [8]:
# The fields "Screenshots" from ds1 and "short_screenshots" from ds2 need to be cleaned first.

# ds1
ds1ss = [] # Flatten into a clean list of URLs
for item in ds1['Screenshots']:
    if not item:
        continue
    ds1ss.extend([url.strip() for url in item.split(",") if url.strip()])

# ds2
ds2ss = [] # Extract image URLs
for elem in ds2['short_screenshots']:
    if not elem:
        continue
    records = elem.split('|') # Split the string by '|'
    for rec in records:
        # Convert string to dict safely
        d = ast.literal_eval(rec)
        ds2ss.append(d['image'])

In [9]:
# This function downloads URL in links as a stream and uploads it directly to the given bucket,
# saving the files with names like image_1.jpg, image_2.png, etc., while preserving their extensions.
def upload_media_from_links(bucket_name, client, links, path="temporal-landing/", prefix="image"):
    for i, url in enumerate(links, start=1):
        if not url:
            continue

        try:
            # Stream download to avoid loading full file in memory
            with requests.get(url, stream=True, timeout=120) as r:
    
                if r.status_code == 404:
                    print(f"Skipped (404 Not Found): {url}") # 404 -> skip
                    continue
                
                r.raise_for_status() # check for HTTP errors
                ext = url.split('.')[-1].split('?')[0] # get file extension
                object_name = f"{path}{prefix}_{i}.{ext}"
                # This streams the request directly to MinIO without creating a full BytesIO object
                client.upload_fileobj(
                    Fileobj=r.raw,
                    Bucket=bucket_name,
                    Key=object_name,
                    ExtraArgs={"ContentType": f"{prefix}/{ext}"}
                )
    
                print(f"Uploaded: {object_name}")
        except requests.exceptions.Timeout: # 
            print(f"Skipped (Timeout): {url}") # Timeout -> skip
        except requests.exceptions.HTTPError as e:
            print(f"HTTP error for {url}: {e}") # Other HTTP errors -> log + skip
        except Exception as e:
            print(f"Failed {url}: {e}") # Other exceptions -> log + skip

In [10]:
# Uploading image files (combining both datasets)
upload_media_from_links("landing-zone", s3, links = 
                         ds1['Header image'] + ds2['background_image'], # + ds2['background_image_additional'] + ds1ss + ds2ss,
                         path="temporal-landing/") # If this process is taking too long, we can just skip the screeshots

Uploaded: temporal-landing/image_1.jpg
Uploaded: temporal-landing/image_2.jpg
Uploaded: temporal-landing/image_3.jpg
Uploaded: temporal-landing/image_4.jpg
Uploaded: temporal-landing/image_5.jpg
Uploaded: temporal-landing/image_6.jpg
Uploaded: temporal-landing/image_7.jpg
Uploaded: temporal-landing/image_8.jpg
Uploaded: temporal-landing/image_9.jpg
Uploaded: temporal-landing/image_10.jpg
Uploaded: temporal-landing/image_11.jpg
Uploaded: temporal-landing/image_12.jpg
Uploaded: temporal-landing/image_13.jpg
Uploaded: temporal-landing/image_14.jpg
Uploaded: temporal-landing/image_15.jpg
Uploaded: temporal-landing/image_16.jpg
Uploaded: temporal-landing/image_17.jpg
Uploaded: temporal-landing/image_18.jpg
Uploaded: temporal-landing/image_19.jpg
Uploaded: temporal-landing/image_20.jpg
Uploaded: temporal-landing/image_21.jpg
Uploaded: temporal-landing/image_22.jpg
Uploaded: temporal-landing/image_23.jpg
Uploaded: temporal-landing/image_24.jpg
Uploaded: temporal-landing/image_25.jpg
Uploaded:

**Uploading Videos**

In [12]:
# Uploading video files
upload_media_from_links("landing-zone", s3, links = 
                         ds1['Movies'][0:30], # It's recommended to upload only a few videos due to MinioIO's storage size
                         path="temporal-landing/", prefix = "video") # Here we are only selecting the first 30 videos

Uploaded: temporal-landing/video_1.mp4
Uploaded: temporal-landing/video_2.mp4
Uploaded: temporal-landing/video_3.mp4
Uploaded: temporal-landing/video_4.mp4
Uploaded: temporal-landing/video_5.mp4
Uploaded: temporal-landing/video_6.mp4
Uploaded: temporal-landing/video_7.mp4
Uploaded: temporal-landing/video_8.mp4
Uploaded: temporal-landing/video_9.mp4
Uploaded: temporal-landing/video_10.mp4
Uploaded: temporal-landing/video_11.mp4
Uploaded: temporal-landing/video_12.mp4
Uploaded: temporal-landing/video_13.mp4
Uploaded: temporal-landing/video_14.mp4
Uploaded: temporal-landing/video_15.mp4
Uploaded: temporal-landing/video_17.mp4
Uploaded: temporal-landing/video_18.mp4
Uploaded: temporal-landing/video_19.mp4
Uploaded: temporal-landing/video_20.mp4
Uploaded: temporal-landing/video_21.mp4
Uploaded: temporal-landing/video_22.mp4
Uploaded: temporal-landing/video_23.mp4
Uploaded: temporal-landing/video_24.mp4
Uploaded: temporal-landing/video_25.mp4
Uploaded: temporal-landing/video_26.mp4
Uploaded: